# <span style="color:#1E90FF;">Data Cleaning and Imputation Process for Cat Breeds Dataset</span>
#

## <span style="color:#FF4500;">1. Initial Data Inspection</span>

In [27]:
import pandas as pd
import numpy as np


In [28]:
# Load the dataset with the correct delimiter and header
file_path = "cat_breeds_dirty.csv"  # Replace with your file path
df = pd.read_csv(file_path, delimiter=';')


In [29]:
df.head()

,Breed,Age_in_years,Age_in_months,Gender,Neutered_or_spayed,Body_length,Weight,Fur_colour_dominant,Fur_pattern,Eye_colour,Allowed_outdoor,Preferred_food,Owner_play_time_minutes,Sleep_time_hours,Country,Latitude,Longitude
0,Angora,0.25,3.0,female,False,19.0,2.0,white,solid,blue,FALSE,wet,46.0,16.0,France,43.296482,5.369780
1,Angora,0.33,4.0,male,False,19.0,2.5,white,solid,blue,FALSE,wet,48.0,16.0,France,43.611660,3.877710
2,Angora,0.50,NaN,NaN,False,20.0,2.8,what does it mean dominant?,solid,green,I never allow my kitty outside!!!!!,wet,41.0,11.0,France,44.837789,-0.579180
3,Ankora,0.50,NaN,NaN,False,21.0,3.0,white,dirty,blue,FALSE,wet,24.0,8.0,France,43.611660,3.877710
4,Angora,0.50,NaN,NaN,NaN,21.0,3.0,red/cream,tabby,green,FALSE,wet,51.0,10.0,france,48.864716,2.349014


- Loaded the dataset and inspected the first few rows to understand its structure.
- Detected that the data was separated by semicolons (`;`) instead of commas, and adjusted the loading method accordingly.
- Verified the presence of a proper header row and set it as the column names.

#
## <span style="color:#FF4500;">2. Data Type Consistency</span>

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1103 entries, 0 to 1102
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Breed                    991 non-null    object 
 1   Age_in_years             1072 non-null   float64
 2   Age_in_months            1066 non-null   float64
 3   Gender                   1036 non-null   object 
 4   Neutered_or_spayed       1050 non-null   object 
 5   Body_length              1077 non-null   float64
 6   Weight                   1077 non-null   float64
 7   Fur_colour_dominant      1090 non-null   object 
 8   Fur_pattern              1055 non-null   object 
 9   Eye_colour               1064 non-null   object 
 10  Allowed_outdoor          1060 non-null   object 
 11  Preferred_food           1082 non-null   object 
 12  Owner_play_time_minutes  1082 non-null   float64
 13  Sleep_time_hours         1062 non-null   float64
 14  Country                 

In [31]:
df = df.convert_dtypes()

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1103 entries, 0 to 1102
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Breed                    991 non-null    string 
 1   Age_in_years             1072 non-null   Float64
 2   Age_in_months            1066 non-null   Int64  
 3   Gender                   1036 non-null   string 
 4   Neutered_or_spayed       1050 non-null   boolean
 5   Body_length              1077 non-null   Int64  
 6   Weight                   1077 non-null   Float64
 7   Fur_colour_dominant      1090 non-null   string 
 8   Fur_pattern              1055 non-null   string 
 9   Eye_colour               1064 non-null   string 
 10  Allowed_outdoor          1060 non-null   string 
 11  Preferred_food           1082 non-null   string 
 12  Owner_play_time_minutes  1082 non-null   Int64  
 13  Sleep_time_hours         1062 non-null   Int64  
 14  Country                 

Summary:
convert_dtypes() improves column data types for consistency, missing value support, and analysis. It's especially helpful after loading messy or mixed-type datasets.

- Converted columns to appropriate data types (e.g., numeric, categorical, boolean) to ensure consistency.
- Checked for columns containing mixed data types and standardized them.

#
## <span style="color:#FF4500;">3. Whitespace and Formatting Cleanup</span>

In [33]:
# Trim whitespace and fix double spaces in string columns

str_cols = df.select_dtypes(include='string').columns
for col in str_cols:
    df[col] = df[col].str.strip().str.replace(r'\s+', ' ', regex=True)

In [34]:
# Standardize categorical values (example: capitalize country names)

if 'Country' in df.columns:
    df['Country'] = df['Country'].str.capitalize()

- Trimmed leading and trailing whitespaces from all string columns.
- Replaced multiple spaces within strings with a single space.
- Standardized categorical values (e.g., capitalizing country names) to avoid duplicates caused by casing differences.

#
## <span style="color:#FF4500;">4. Handling Missing Values</span>

In [35]:
# Identify columns with missing values and their data types

missing_info = df.isna().sum()
missing_info = missing_info[missing_info > 0].sort_values(ascending=False)

missing_info

Breed                      112
Country                     75
Gender                      67
Latitude                    61
Longitude                   61
Neutered_or_spayed          53
Fur_pattern                 48
Allowed_outdoor             43
Sleep_time_hours            41
Eye_colour                  39
Age_in_months               37
Age_in_years                31
Body_length                 26
Weight                      26
Preferred_food              21
Owner_play_time_minutes     21
Fur_colour_dominant         13
dtype: int64

- Identified columns with missing values and their counts.

#
## <span style="color:#FF4500;">5. Imputation Strategy</span>


In [36]:
# Replace unclear values with NaN for imputation

if 'Fur_colour_dominant' in df.columns:
    df['Fur_colour_dominant'] = df['Fur_colour_dominant'].replace('what does it mean dominant?', pd.NA)

In [37]:
# Define columns by type for imputation

numerical_cols = df.select_dtypes(include=['Int64', 'Float64']).columns.tolist()
categorical_cols = df.select_dtypes(include=['string', 'boolean']).columns.tolist()

In [38]:
# Fill numerical columns with median

for col in numerical_cols:
    median_value = df[col].median()
    df[col] = df[col].fillna(median_value)


In [39]:
# Fill categorical columns with mode

for col in categorical_cols:
    mode_value = df[col].mode(dropna=True)
    if not mode_value.empty:
        df[col] = df[col].fillna(mode_value[0])

- For numerical columns (e.g., Age, Weight, Body Length):
  - Filled missing values with the **median** of the respective column to reduce the impact of outliers.
- For categorical columns (e.g., Breed, Gender, Fur Color):
  - Filled missing values with the **mode** (most frequent value) of the respective column to maintain category consistency.
- For columns with unclear or invalid entries (e.g., "what does it mean dominant?"):
  - Replaced such entries with missing values before applying imputation.

#
## <span style="color:#FF4500;">6. Final Verification and Export</span>

In [40]:
# Verify no missing values remain

missing_after_fill = df.isna().sum().sum()
print(f'Total missing values after imputation: {missing_after_fill}')

Total missing values after imputation: 0


In [41]:
# Save the cleaned and imputed dataset

output_path = 'cleaned_dataset.csv'  
df.to_csv(output_path, index=False)
print(f'Cleaned dataset saved to {output_path}')

Cleaned dataset saved to cleaned_dataset.csv


- Verified that no missing values remained after imputation.
- Saved the cleaned and imputed dataset as a CSV file for further analysis or sharing.


#
## <span style="color:#32CD32;">Summary</span>


This process ensures that the dataset is clean, consistent, and ready for analysis or modeling. Proper handling of missing data and data types improves the quality and reliability of insights derived from the data.
